# Week-5: Create a simple convolution network using tf.contrib.keras

## Get the input data (only the first 'n' labels) from MNIST

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST", one_hot=True)

Extracting MNIST\train-images-idx3-ubyte.gz
Extracting MNIST\train-labels-idx1-ubyte.gz
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz


## Prepare Session

In [2]:
import numpy as np

In [3]:
import tensorflow as tf
from tensorflow.contrib import keras
# from tensorflow.contrib.keras.layers import *

## Define a sequential model

In [4]:
inputs = keras.layers.Input(shape=(784,))

# Reshape layer, takes (-1, 784) as input, (-1, 28, 28, 1) as output
reshape_layer = keras.layers.Reshape((28, 28, 1), input_shape=(784, ))(inputs)


# Convolution 2D layer, takes (-1, 28, 28, 1) as input, (-1, 13, 13, 32) as output
conv_2D_layer = keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=(2, 2), activation="relu")(reshape_layer)

# Flatten layer, takes (-1, 13, 13, 32) as input, (-1, 13*13*32) as output
flatten_layer = keras.layers.Flatten()(conv_2D_layer)

# Dense layer, takes (-1, 5408) as input, (-1, 100) as output
dense_layer = keras.layers.Dense(units=100, activation="relu")(flatten_layer)

# Softmax layer
softmax_layer = keras.layers.Dense(units=10, activation="softmax")(dense_layer)


## compile the model

In [5]:
model = keras.models.Model(inputs = inputs, outputs = softmax_layer)

model.compile(optimizer = keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## fit the model

In [6]:
x_train = mnist.train.images
y_train = mnist.train.labels

In [7]:
model.fit(x_train, y_train,
          epochs=2,
          batch_size=128)

Epoch 1/2
55000/55000 [==============================] - 9s - loss: 0.2925 - acc: 0.9180     
Epoch 2/2
55000/55000 [==============================] - 8s - loss: 0.0941 - acc: 0.9726     


## add a node as input

In [8]:
addl_node = keras.layers.Input(shape=(1,), name = 'addl_node')

new_layer = keras.layers.concatenate([dense_layer, addl_node])

new_dense_layer = keras.layers.Dense(units=100, activation="relu")(new_layer)

new_softmax_layer = keras.layers.Dense(units=10, activation="softmax")(new_dense_layer)

model = keras.models.Model(inputs=[inputs, addl_node], outputs=new_softmax_layer)

model.compile(optimizer = keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
x_train = mnist.train.images
x_addl_node = np.array([1] * 55000)
y_train = mnist.train.labels

In [10]:
model.fit([x_train,x_addl_node], y_train,
          epochs=2,
          batch_size=128,
          callbacks=[keras.callbacks.TensorBoard(log_dir='./log_new/', histogram_freq=0, write_graph=True, write_images=True)])

Epoch 1/2
55000/55000 [==============================] - 8s - loss: 0.1737 - acc: 0.9510     
Epoch 2/2
55000/55000 [==============================] - 10s - loss: 0.0560 - acc: 0.9827    
